In [1]:
import os
import glob
import datetime
import unidecode

import numpy as np
import pandas as pd

from collections.abc import Sequence
from functools import cache
from typing import Any

from tqdm.notebook import tqdm

from engineFD import EngineFD
from engineDK import EngineDK
from filing import Filing

In [2]:
pd.options.display.max_rows = 350
pd.options.display.max_columns = 50
pd.options.display.max_colwidth = 75

In [3]:
class PastPerformances:
    def clean_name(cls, name: str) -> str:
        """
        Standardizes name across PFR, FD, DK
        """
        return ' '.join(name.split(' ')[:2]).replace('.', '')

    def flatten(cls, nestedSeq: Sequence[Sequence[Any,...], ...], **kwargs) -> list[Any,...]:
        """
        Takes 2d sequence and returns all values in 1d
        Example: [(a,b,c), (a,y,z), (a,b,z)] -> [a, b, c, a, y, z, a, b, z]
        TODO: kwargs to add functinoality
            - unique: [(a,b,c), (a,y,z), (a,b,z)] -> [a, b, c, y, z]
            - counts dict -> {a: 3, b: 2, z: 2, c: 1, z: 1} (recursive)
            - etc
        """
        if kwargs.get('unique', False):
            return set(cls.flatten(nestedSeq))

        return [element for innerSeq in nestedSeq for element in innerSeq]

    def second_min(cls, seq: Sequence[int|float, ...]) -> int|float:
        """
        Takes numerical sequence and returns second minimum value
        """
        minimum = min(seq)
        return min([val for val in seq if val != minimum])

    def third_min(cls, seq: Sequence[int|float, ...]) -> int|float:
        """
        Takes numerical sequence and returns third minimum value
        """
        minimum = min(seq)
        secondmin = cls.second_min(seq)
        return min([val for val in seq if val not in (minimum, secondmin)])


    def second_max(cls, seq: Sequence[int|float, ...]) -> int|float:
        """
        Takes numerical sequence and returns second maximum value
        """
        maximum = max(seq)
        return max([val for val in seq if val != maximum])

    def third_max(cls, seq: Sequence[int|float, ...]) -> int|float:
        """
        Takes numerical sequence and returns third maximum value
        """
        maximum = max(seq)
        secondmax = cls.second_max(seq)
        return max([val for val in seq if val not in (maximum, secondmax)])

    def get_contest_files_with_n_games(cls, *, site: str, contest_files: list[str,...], n_games: int, **kwargs) -> list[str,...]:
        """
        Takes a list of contest files and an integer as an input and searches all input files for those with that number of games
        As of right now, only compatible with 2022-2023 season
        Defaults:
            - site: FanDuel
            - mode: main-slate
            - TODO: option for late slate
        """
    
        # Cushion added to include similar sized slates
        # n_teams = range((n_games-1)*2, (n_games+1)*2 + 1) if n_games > 4 else (n_games*2, )
        n_teams = (n_games*2, )
    
        team_column = 'Team' if site == 'fanduel' else 'TeamAbbrev'
        
        n_games_files = [
            file for file in contest_files
            if len(pd.read_csv(file)[team_column].drop_duplicates()) in n_teams
        ]
    
        return n_games_files

    
    @cache
    def extract_date(self, file: str):
        """
        Extracts the date from a file
        """
        return file.split('/')[-1].split('.')[0] #.replace('') #.replace('a', '').replace('b', '').replace('-late', '')

    def __init__(self, *, n_games: int, **kwargs) -> None:
        """
        Creates instance of class to analyze past performances
        Defaults if not given:
            - site: FanDuel
            - contest: Main-Slate
            - season: 2022-2023 Season
        """

        self.n_games = n_games

        self.min_fpts = kwargs.get('min_fpts', 45.0)
        self.min_fpts_1k = kwargs.get('min_fpts_1k', 6.0)

        if n_games < 5:
            self.min_fpts = 25.0
            self.min_fpts_1k = 4.0

        if kwargs.get('late_slate', False):
            self.min_fpts = 20.0
            self.min_fpts_1k = 6.0
        
        self.site = kwargs.get('site', 'fanduel')
        self.contest = kwargs.get('contest', 'main-slate')

        self.year = kwargs.get('year', 2022)
        self.season = f'{self.year}-{self.year+1}'

        self.filing = Filing(self.season)

        # All contest files saved for year and mode
        self.contest_files = glob.glob(os.path.join(self.filing.season_dir, 'contest-files', self.site, self.contest) + '/*.csv')
        
        # All boxscores for year
        self.boxscore_files = glob.glob(self.filing.boxscores_dir + '/*.csv')


        # Files will create optimals for

        
        self.n_games_files = glob.glob(os.path.join(self.filing.season_dir, 'contest-files', self.site, f'{self.n_games}-games') + '/*.csv')

        n_games_p1_path = os.path.join(self.filing.season_dir, 'contest-files', self.site, f'{self.n_games+1}-games')
        self.n_games_p1_files = glob.glob(n_games_p1_path + '/*.csv') if os.path.exists(n_games_p1_path) else list()

        n_games_m1_path = os.path.join(self.filing.season_dir, 'contest-files', self.site, f'{self.n_games-1}-games')
        self.n_games_m1_files = glob.glob(n_games_m1_path + '/*.csv') if os.path.exists(n_games_m1_path) else list()
        
        
        return None


    def get_boxscores_on_date(self, date: str) -> list[str,...]:
        """
        Returns all file paths for boxscores from given date
        As of rn, boxscores are saved for individual teams rather than all in one date (TODO: change?)
        Will be used in conjuction with pd.concat

        """
        return [file for file in self.boxscore_files if date in file]

    
    def load_boxscore_data(self, files: list[str,...]) -> dict[str, pd.DataFrame]:
        """
        Takes list of paths to boxscores CSV files as input
        If error occuring with no items to concat, make sure current date's contest is not in n-games folder
        """
        return {
            self.extract_date(file): (pd
                                      .concat([pd.read_csv(date_file) for date_file in self.get_boxscores_on_date(self.extract_date(file))])
                                      .assign(
                                          fpts=lambda df_: df_.fd_fpts if self.site == 'fanduel' else df_.dk_fpts,
                                          # name=lambda df_: unidecode.unidecode(df_.name.str)
                                      )
                                      [['date', 'name', 'team', 'opp', 'starter', 'mp', 'pace', 'fpts']]
                                     )
            for file in files
        }

    def load_data(self):
        """
        Creates dataframes that can be plugged into optimizer for all dates where self.n_games were played
        Need FPTS and possible other info from boxscores csv files
        Need Position and Salary from contest csv files
        Therefore need to merge the two corresponding dfs for each date
        """

        # Combined boxscores for all dates where self.n_games were played
        # boxscore_dfs = self.load_boxscore_data(self.n_games_files)

        boxscore_dfs = {
            **self.load_boxscore_data(self.n_games_files),
            # **self.load_boxscore_data(self.n_games_p1_files),
            # **self.load_boxscore_data(self.n_games_m1_files)
        } if self.n_games != 2 else self.load_boxscore_data(self.n_games_files)


        # Contest files for dates where self.n_games were played
        #Issue here? Trying to target late slate contests
        # contest_files = [file for file in self.contest_files if self.extract_date(file) in boxscore_dfs]

        # Fixed: Changed location from which reading contest-files, renamed 2 game contests to be just date (to match boxscore) and assigned to new directory in sandbox.ipynb
        # Above ^(commented out)^ because self.contest_files refers to directory with all contest files, in the case of wanting to target 2 game contests, need to target files that originally have -late in name, was instead reading main-slate contests
        contest_files = [file for file in self.n_games_files if self.extract_date(file) in boxscore_dfs]

        # contest_files = sum([
        #     # Contest files for dates where self.n_games were played
        #     [file for file in self.contest_files if self.extract_date(file) in self.load_boxscore_data(self.n_games_files)],
            
        #     # Contest files for dates where self.n_games+1 were played
        #     [file for file in self.contest_files if self.extract_date(file) in self.load_boxscore_data(self.n_games_p1_files)],
            
        #     # Contest files for dates where self.n_games-1 were played
        #     [file for file in self.contest_files if self.extract_date(file) in self.load_boxscore_data(self.n_games_m1_files)]
                                                                                                       
        # ], list())

        contest_columns = {
            'draftkings': ['Name', 'Salary', 'Position', 'TeamAbbrev'],
            'fanduel': ['Nickname', 'Salary', 'Position', 'Team']
        }

        contest_dfs = {
            self.extract_date(file): (pd
                                      .read_csv(file)
                                      [contest_columns[self.site]]
                                      .set_axis(['name', 'salary', 'pos', 'team'], axis=1)
                                      .assign(date=self.extract_date(file))
                                     )
            for file in contest_files
        }

        # Create dataframes able to be plugged into optimizer
        self.optimizer_dfs = dict()

        # Identical keys so doesn't matter
        for date in contest_dfs:
            # Naming could be a little confusing, TODO: fix this
            boxscore_df, contest_df = boxscore_dfs[date], contest_dfs[date]

            # Sanitize names from boxscores
            boxscore_df['name'] = boxscore_df['name'].map(lambda name: unidecode.unidecode(name))

            # boxscore_df['game'] = boxscore_df[['team', 'opp']]

            boxscore_df['index_'] = boxscore_df[['date', 'name']].apply(lambda row: f'{row.iloc[0]}_{row.iloc[1]}')
            contest_df['index_'] = contest_df[['date', 'name']].apply(lambda row: f'{row.iloc[0]}_{row.iloc[1]}')

            # number of games on slate
            # slate_games = int()
        
            boxscore_df = boxscore_df.set_index('index_')
            contest_df = contest_df.set_index('index_')

            # Creates optimizer df with cutoff for fpts because will probably be too large for optimizer
            
            # Issues are with file naming
            # If players are greater than min_fpts, have to be at least 5.0 fpts_1k
            # if len(contest_df['team'].drop_duplicates()) == self.n_games * 2:
            self.optimizer_dfs[date] = (boxscore_df
                                        .merge(contest_df.loc[:, ['name', 'salary', 'pos']])
                                        .dropna() # ??
                                        .set_index('name')
                                        .sort_values('fpts', ascending=False)
                                        .assign(
                                            fpts_1k=lambda df_: 1_000 * df_.fpts / df_.salary,
                                            slate_games=len(contest_df['team'].drop_duplicates()) // 2
                                        )
                                        .pipe(lambda df_: df_.loc[((df_['fpts'] >= self.min_fpts)) | ((df_['fpts'] < self.min_fpts) & (df_['fpts_1k'] >= self.min_fpts_1k))])
                                        # .pipe(lambda df_: df_.loc[((df_['fpts'] >= self.min_fpts) & (df_['fpts_1k'] >= 5.0)) | ((df_['fpts'] < self.min_fpts) & (df_['fpts_1k'] >= self.min_fpts_1k))])
                                        .head(25)
                                        .round(3)
                                       )

        self.optimizer_dfs = {date: date_df for date, date_df in self.optimizer_dfs.items() if not date_df.empty}
                    
        
        return None


    def create_optimal(self, data: pd.DataFrame):
        """
        Creates optimal for specific date
        data is self.optimizer_dfs[date]
        """
        engine = {'draftkings': EngineDK, 'fanduel': EngineFD}[self.site](data)

        optimal = engine.create_lineups(top_n=10).reset_index(drop=True)

        if 'lineup' not in optimal.columns:
            optimal['lineup'] = optimal[engine.labels].apply(tuple, axis=1)
            
        optimal['lineup'] = optimal['lineup'].map(lambda names: engine.checker.order(names))

        optimal['n_teams'] = optimal['lineup'].map(lambda names: len(set([engine.checker.pvalue(name, 'team') for name in names])))
        optimal['n_starters'] = optimal['lineup'].map(lambda names: sum([engine.checker.pvalue(name, 'starter') for name in names]))
        optimal['n_games'] = optimal['lineup'].map(lambda names: len(set([engine.checker.pvalue(name, 'game') for name in names])))


        optimal['teams'] = optimal['lineup'].map(lambda names: '-'.join([engine.checker.pvalue(name, 'team') for name in names]))
        optimal['games'] = optimal['lineup'].map(lambda names: ':'.join([engine.checker.pvalue(name, 'game') for name in names]))

        optimal['distro'] = optimal['teams'].map(lambda teams: tuple(sorted([teams.split('-').count(team) for team in set(teams.split('-'))])))
        optimal['game-distro'] = optimal['games'].map(lambda games: tuple(sorted([games.split(':').count(team) for team in set(games.split(':'))])))

        optimal['salaries'] = optimal['lineup'].map(lambda names: tuple(sorted([engine.checker.pvalue(name, 'salary') for name in names])))
        optimal['minutes'] = optimal['lineup'].map(lambda names: tuple(sorted([engine.checker.pvalue(name, 'mp') for name in names])))

        # optimal['mt1_player'] = optimal['teams'].map(lambda teams: tuple([team for team in teams.split('-') if teams.count(team) > 1]))
        # optimal['teammates'] = optimal[['lineup', 'mt1_player']].apply(lambda row: tuple([name for name in row.iloc[0] if engine.checker.pvalue(name, 'team') in row.iloc[1]]), axis=1)
        # optimal['teammate_pos'] = optimal['teammates'].map(lambda names: tuple([engine.checker.pvalue(name, 'pos') for name in names]))
        
        optimal['C-salary'] = optimal['C'].map(lambda name: engine.checker.pvalue(name,'salary'))

        
        # optimal['min_salary'] = optimal['lineup'].map(lambda names: min([engine.checker.pvalue(name, 'salary') for name in names]))
        # optimal['min2_salary'] = optimal['lineup'].map(lambda names: self.second_min([engine.checker.pvalue(name, 'salary') for name in names]))
        # optimal['min3_salary'] = optimal['lineup'].map(lambda names: self.third_min([engine.checker.pvalue(name, 'salary') for name in names]))
        
        # optimal['max_salary'] = optimal['lineup'].map(lambda names: max([engine.checker.pvalue(name, 'salary') for name in names]))
        # optimal['max2_salary'] = optimal['lineup'].map(lambda names: self.second_max([engine.checker.pvalue(name, 'salary') for name in names]))
        # optimal['max3_salary'] = optimal['lineup'].map(lambda names: self.third_max([engine.checker.pvalue(name, 'salary') for name in names]))
        

        return optimal.drop('lineup', axis=1)


    def create_optimals(self):

        if not hasattr(self, 'optimizer_dfs'):
            self.load_data()

        # Did not do dictcomp so progress_bar possible
        # self.optimals = dict()

        # for date, date_data in tqdm(self.optimizer_dfs.items()):
        #     self.optimals[date] = self.create_optimal(date_data)

        self.optimals = {date: self.create_optimal(date_data) for date, date_data in tqdm(self.optimizer_dfs.items())}


        return None

    def view_optimal(self, date: str) -> pd.DataFrame:

        if hasattr(self, 'optimals'):
            return self.optimals[date]

        if not hasattr(self, 'optimizer_dfs'):
            self.load_data()

        return self.create_optimal(self.optimizer_dfs[date])

In [4]:
# YESTERDAY = datetime.datetime.strftime((datetime.datetime.now() - datetime.timedelta(days=1)), '%Y-%m-%d')
# pp23 = PastPerformances(year=2023, n_games=4)

In [5]:
pp23 = PastPerformances(year=2023, n_games=2, site='draftkings')

In [6]:
pp23.load_data()

In [7]:
# pp23.optimizer_dfs #[YESTERDAY]
len(pp23.optimizer_dfs)

22

In [ ]:
pp23.create_optimals()
# pp23.optimals #[YESTERDAY]

  0%|          | 0/22 [00:00<?, ?it/s]

In [ ]:
optimals = (pd
            .concat(pp23.optimals.values())
            .drop(['teams', 'minutes', 'games'], axis=1)
            # .pipe(lambda df_: df_.loc[df_.index.isin(range(5))])
           )

In [ ]:
optimals

In [ ]:
optimals.shape

In [ ]:
def report() -> None:
    """
    Reports outcome of optimals crunch
    """
    sample_size = len(pp23.optimizer_dfs)
    target_rows = sample_size * 10

    rows = optimals.shape[0]
    output = list()
    
    try:
        assert(rows == target_rows)
        output.append('Optimals successful for all samples')

    except AssertionError:
        missing = target_rows - rows
        days_missing = missing // 10
        output.append(f'Missing {days_missing} optimal slate{"" if days_missing == 1 else "s"}')

    print(*output, sep='\n')
    return

report()

In [ ]:
def percentages(s: pd.Series) -> pd.Series:
    """
    Takes a series and returns edited values_counts
    """
    return s.value_counts(normalize=True).round(3) * 100

In [ ]:
percentages(optimals['salary'])

In [ ]:
optimals['max-salary'] = optimals['salaries'].map(lambda sals: max(sals))
optimals['max-salary-2'] = optimals['salaries'].map(lambda sals: sorted(sals)[-2])
optimals['max-salary-3'] = optimals['salaries'].map(lambda sals: sorted(sals)[-3])

optimals['sum-max-2-sals'] = optimals['max-salary'] + optimals['max-salary-2']
optimals['sum-max-3-sals'] = optimals['sum-max-2-sals'] + optimals['max-salary-3']

In [ ]:
percentages(optimals['max-salary'])

In [ ]:
percentages(optimals['max-salary-2'])

In [ ]:
percentages(optimals['max-salary-3'])

In [ ]:
percentages(optimals['sum-max-2-sals'])

In [ ]:
percentages(optimals['sum-max-3-sals'])

In [ ]:
percentages(optimals['n_games'])

In [33]:
percentages(optimals['distro']) #.sum()

(1, 2, 3, 3)    41.8
(1, 2, 2, 4)    20.3
(2, 2, 2, 3)    16.4
(2, 3, 4)       10.0
(1, 1, 3, 4)     7.9
(3, 3, 3)        2.7
(1, 4, 4)        0.9
Name: distro, dtype: float64

In [34]:
percentages(optimals['game-distro'])

(4, 5)    63.6
(3, 6)    32.1
(2, 7)     4.2
Name: game-distro, dtype: float64

In [35]:
percentages(optimals['n_teams'])

4    86.4
3    13.6
Name: n_teams, dtype: float64

In [ ]:
percentages(optimals['C-salary'])

In [ ]:
df_sals = (pd
           .DataFrame(optimals['salaries'].value_counts(normalize=True))
           .reset_index()
           .set_axis(['sals', '%'], axis=1)
          )

In [ ]:
df_sals['max'] = df_sals['sals'].map(lambda sals: max(sals))
df_sals['min'] = df_sals['sals'].map(lambda sals: min(sals))

df_sals = df_sals.sort_values('max', ascending=False)

In [ ]:
df_sals['lt4k'] = df_sals['sals'].map(lambda sals: len([sal for sal in sals if sal < 4_000]))

In [ ]:
df_sals['lt5k'] = df_sals['sals'].map(lambda sals: len([sal for sal in sals if sal < 5_000]))

In [ ]:
df_sals['lt6k'] = df_sals['sals'].map(lambda sals: len([sal for sal in sals if sal < 6_000]))

In [ ]:
df_sals['lt7k'] = df_sals['sals'].map(lambda sals: len([sal for sal in sals if sal < 7_000]))

In [ ]:
percentages(df_sals['lt4k'])

In [ ]:
percentages(df_sals['lt5k'])

In [ ]:
percentages(df_sals['lt6k'])

In [ ]:
percentages(df_sals['lt7k'])

In [ ]:
percentages(df_sals['max'])

In [ ]:
percentages(df_sals['min'])

In [ ]:
min_sals = df_sals['min']

In [ ]:
not_4kmin = [sal for sal in min_sals if sal < 4_000 or sal >= 5_000]

In [ ]:
min_sals_perc = percentages(df_sals['min'])

In [ ]:
min_sals_perc[list(set(not_4kmin))].sum()

In [ ]:
df_sals['sals'] = df_sals['sals'].map(lambda distro: tuple([str(sal)[:-3] for sal in distro]))

In [ ]:
df_sals['minsal'] = df_sals['sals'].map(lambda sals: min([int(sal_) for sal_ in sals]))
percentages(df_sals['minsal'])
# df_sals

In [ ]:
for str_n in [str(n) for n in (3,4,5,6,7,8,9,10,11)]:
    df_sals[str_n] = df_sals['sals'].map(lambda sals: int(str_n in sals))
    df_sals[f'{str_n}-counts'] = df_sals['sals'].map(lambda sals: sals.count(str_n))

In [ ]:
df_sals['3-4-counts'] = df_sals['sals'].map(lambda sals: sum([sals.count('3'), sals.count('4')]))

In [ ]:
percentages(df_sals['3-4-counts'])

In [ ]:
count_cols = [col for col in df_sals.columns if 'counts' in col]

In [ ]:
percentages(df_sals['3-counts'])

In [ ]:
percentages(df_sals['4-counts'])

In [ ]:
percentages(df_sals['5-counts'])

In [ ]:
percentages(df_sals['6-counts'])

In [ ]:
percentages(df_sals['7-counts'])

In [ ]:
percentages(df_sals['8-counts'])

In [ ]:
percentages(df_sals['9-counts'])

In [ ]:
percentages(df_sals['10-counts'])

In [ ]:
percentages(optimals['max_salary'])

In [ ]:
percentages(optimals['min_salary'])

In [ ]:
percentages(optimals['distro'])